<a href="https://colab.research.google.com/github/angui100/Tensorflow_certificate/blob/main/artificial_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_file = '/content/drive/My Drive/datasource/Churn_Modelling.csv'
data_set = pd.read_csv( data_file )
print(data_set)

Data analyze part
## 1. independent variables

In [ ]:
X = data_set.iloc[:, 3:-1].values
print(X)

## 2. Dependent variables

In [ ]:
y = data_set.iloc[:, -1].values
print(y)

## 3. Need to fix any missing data?

### Encoding categorical data

Label Encoding the "Gender" column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

print(X)


One Hot Encoding the "Geography" column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer( transformers=[('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)

print(type(X))
print(X)


## Convert the numpy.ndarray to numpy array ? Answer: Not needed here.

In [ ]:
X = np.array(X) # ? Really need to convert to Numpy array
print(type(X))

### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
print(X_train)

In [ ]:
print(len(X_train))
print(X[-1, :])

In [ ]:
print(X_test[-1, :])

In [ ]:
print(len(X_test))

### Feature Scaling (Very Important)
Need to determine what are the columns of data need to be standarlized in this model.

In [ ]:
print(X_test[-1, :]) #The las row of data
print(X_test[:, 4:])

From the last row of data, we do see there are 3 columns may need to be standarlized for this model. Which are indexes [4], [8], [12]
<br>In order to standarlize the related data, do we need to rearrange the columns of the dataframe ???

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
#X_train[:, 4:] = sc.fit_transform(X_train[:, 4:])
X_train = sc.fit_transform(X_train)

In [ ]:
print(X_train[-1:, :])

In [ ]:
print(X_train)

In [ ]:
print(X_train[1, :])

In [ ]:
X_test = sc.transform(X_test)
print(X_test)

In [ ]:
print(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [ ]:
ann.add( tf.keras.layers.Dense(units=6, activation='relu' ))

### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense( units=6, activation='relu'))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the ANN on the Training set

In [ ]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)
ann.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test), validation_steps=20)

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

In [ ]:
x =[ [1.0, 0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000] ]
x = sc.transform(x)
#x = [[1, 0, 0, 6.e+02 1.e+00 4.e+01 3.e+00 6.e+04 2.e+00 1.e+00 1.e+00 5.e+04]]
#print(x)
#x = np.asarray(x).astype(np.float32)
#print(x)
#x = sc.transform([[x]])
print(ann.predict(x) )
print(ann.predict(x) > 0.5)

**Solution**

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
print(y_pred)
y_pred = (y_pred > 0.5)
print(y_pred)

In [ ]:
print(np.concatenate( (y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

In [ ]:
print(y_pred.reshape(len(y_pred), 1))

In [ ]:
print(y_test.reshape(len(y_test), 1))

### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
accuracy_score(y_test, y_pred)